# Condensation

Condensation, is the first process where this framework we have been building up is applied. Here we need to account for the gas phase, and the particle phase. Then ensure that the partial pressures of species at the surface of the particle are equal to the partial pressure in the gas.


### Core Concepts:

- **Runnable**: An abstract base class defining the interface for aerosol transformation processes.
  - Here and aerosol object is passed to the process, and the process is expected to modify the aerosol object in place, returning the modified object.
- **MassCondensation**: A concrete class implementing the RunnableProcess interface for the condensation process. Is an implementation of a `Runnable` process that adds mass to the aerosol object based on the partial pressures of the gas phase and the particle phase. Then removes the mass from the gas phase.

### Inputs:

The inputs get a bit complicated here, but in the future we will build factories to handle the creation of these objects and use builder patterns to change units and verify inputs. For now, we will just create them manually.


In [1]:
import numpy as np
from particula.next.gas_species import GasSpeciesBuilder
from particula.next.particle import Particle, particle_strategy_factory
from particula.next.aerosol import Aerosol
from particula.next.gas_vapor_pressure import vapor_pressure_factory
from particula.next.particle_activity import particle_activity_strategy_factory
from particula.next.surface import surface_strategy_factory
from particula.next.particle_process import MassCondensation
from particula.next.condensation import CondensationIsothermal

In [2]:
# Create a Gas instance
# Define the coefficients for Butanol using the Antoine equation.
butanol_coefficients = {
    'latent_heat': 267.0e3,  # check these values, they seem too high, or what latent heat should be used is wrong.
    'temperature_initial': 298.15,
    'pressure_initial': 0.58 * 1e3}
butanol_clausius = vapor_pressure_factory(
    strategy='clausius_clapeyron', **butanol_coefficients)
styrene_coefficients = {
    'latent_heat': 42e3,
    'temperature_initial': 298.15,
    'pressure_initial': 667}
styrene_clausius = vapor_pressure_factory(
    strategy='clausius_clapeyron', **styrene_coefficients)
water_buck = vapor_pressure_factory(
    strategy='water_buck')

# set RH at 10%
water_concentration = water_buck.saturation_concentration(0.017, 298.15)*0.1

# gas species
molar_mass = np.array([0.018, 0.074121, 104.15e-3])
vapor_pressure = [water_buck, butanol_clausius, styrene_clausius]
concentration = np.array([water_concentration, 2e-6, 1e-9])
names = np.array(["water", "butanol", "styrene"])
gas_species = GasSpeciesBuilder() \
    .name(names) \
    .molar_mass(molar_mass) \
    .vapor_pressure_strategy(vapor_pressure) \
    .condensable(True) \
    .concentration(concentration) \
    .build()


# Accessing dynamically attached methods and properties
# Example method call, assuming such a method exists
print(f"Gas mass: {gas_species}")

Gas mass: ['water' 'butanol' 'styrene']


In [11]:
# particle distribution speciated mass, single mode
particle_masses = np.array([0, 4.1e-20, 4.1e-20])  # kg
particle_density = np.array([1.0e3, 1.4e3, 1.4e3])  # kg/m^3
particle_concentration = np.ones_like(particle_masses) * 1e6

# particle activity strategy, or how to treat the mixing of species, kappa, ideal mixing, etc.
particle_activity_kwargs = {
    'kappa': np.array([0.0, 0.2, 0.1]),
    'molar_mass': molar_mass,
    'density': particle_density}
particle_activity = particle_activity_strategy_factory(
    strategy='kappa', **particle_activity_kwargs)

# particle surface strategy, or how to treat the surface properties, surface tension mixing rule, and Kelvin effect
particle_surface_kwargs = {
    'surface_tension': np.array([0.07, 0.03, 0.03]),
    'density': particle_density}
particle_surface = surface_strategy_factory(
    strategy_type='volume', **particle_surface_kwargs)

particle = Particle(
    strategy=particle_strategy_factory("speciated_mass_moving_bin"),
    activity=particle_activity,
    surface=particle_surface,
    distribution=particle_masses,
    density=particle_density,
    concentration=particle_concentration
)

# Particle,
print(f"Particle mass [kg]: {particle.get_mass()}")
print(f"Particle radius [m]: {particle.get_radius()}")

Particle mass [kg]: [0.0e+00 4.1e-20 4.1e-20]
Particle radius [m]: [0.00000000e+00 1.91215196e-08 1.91215196e-08]


## Condensation Process

With the gas phase and particle phase defined, we can now define the condensation process. This process is defined by the following equation: